# 🐳 MCP Containers Demo

## Deploy FastMCP servers on Azure Container Apps with APIM & API Center

This lab deploys **Python [FastMCP](https://github.com/jlowin/fastmcp) servers** as containers in **Azure Container Apps (ACA)**, proxied through **Azure API Management**, and registered in **Azure API Center** for discoverability.

### What you'll learn

1. **Deploy 3 MCP containers** — Weather, Product Catalog, Order Service  
2. **Build & push images** — using ACR Tasks (no local Docker needed)  
3. **Test MCP servers** — directly on ACA + through APIM gateway  
4. **Add a 4th MCP** — Calculator, demonstrating incremental scale-out  
5. **Discover MCP servers** — search & browse via API Center  
6. **Monitor & observe** — Application Insights diagnostics and tracing  
7. **End-to-end agent workflow** — AI agent that chains MCP tools together  

### Architecture

```mermaid
graph TB
    Client[AI Agent / MCP Client] --> APIM[Azure API Management<br/>Streamable MCP Gateway]
    APIM -->|/weather-mcp/mcp| W[☁️ Weather]
    APIM -->|/catalog-mcp/mcp| C[📦 Catalog]
    APIM -->|/order-mcp/mcp| O[🛒 Order]
    APIM -->|/calculator-mcp/mcp| K[🧮 Calculator]
    subgraph ACA[Azure Container Apps Environment]
        W
        C
        O
        K
    end
    ACR[Container Registry] -.->|images| ACA
    APIC[API Center] -.->|discovery| APIM
    AppInsights[Application Insights] -.->|telemetry| APIM
```

### Prerequisites

- [Python 3.12 or later](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor + RBAC Administrator roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in

▶️ Click `Run All` to execute all steps sequentially, or run `Step by Step`...

## Part 1 — Deploy Infrastructure

<a id='0'></a>
### 0️⃣ Initialize notebook variables

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

subscription_id = utils.get_current_subscription()
deployment_name = "mcp-containers"
resource_group_name = f"rg-lab-{deployment_name}"
resource_group_location = "uksouth"

apim_sku = 'Basicv2'
apim_name = "apim-mcp-containers"

apic_location = "uksouth"
apic_service_name_prefix = "apic"

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Deploy infrastructure with Bicep

[demo-mcp-containers.bicep](demo-mcp-containers.bicep) provisions all resources in a single deployment:

| # | Resource | Purpose |
|---|----------|---------|
| 1 | Log Analytics Workspace | Central logging for ACA and APIM |
| 2 | Application Insights | Monitoring, tracing & diagnostics |
| 3 | API Management (Basicv2) | Unified MCP gateway proxy |
| 4 | API Center | MCP server discovery catalog |
| 5 | Azure Container Registry | Container image store |
| 6 | ACA Environment | Serverless container runtime |
| 7 | 4× Container Apps | Weather, Catalog, Order, Calculator MCP servers |
| 8 | 4× APIM MCP APIs | Streamable MCP proxy routes |
| 9 | 4× App Insights Diagnostics | Verbose tracing per MCP API |
| 10 | 4× API Center registrations | MCP discoverability (API, version, definition, deployment) |
| 11 | MCP Insights Dashboard | Azure Portal monitoring dashboard |

In [ ]:
# Create the resource group if it doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": {"value": apim_sku},
        "apimName": {"value": apim_name},
        "apicLocation": {"value": apic_location},
        "apicServiceNamePrefix": {"value": apic_service_name_prefix},
    }
}

# Write the parameters to a file
with open('params-containers.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

# Run the Bicep deployment
output = utils.run(
    f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file demo-mcp-containers.bicep --parameters params-containers.json",
    f"Deployment '{deployment_name}' succeeded",
    f"Deployment '{deployment_name}' failed"
)

<a id='2'></a>
### 2️⃣ Retrieve deployment outputs

In [ ]:
# Obtain all outputs from the deployment
output = utils.run(
    f"az deployment group show --name {deployment_name} -g {resource_group_name}",
    f"Retrieved deployment: {deployment_name}",
    f"Failed to retrieve deployment: {deployment_name}"
)

if output.success and output.json_data:
    apim_service_name = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service Name')
    apim_gateway_url = utils.get_deployment_output(output, 'apimGatewayUrl', 'APIM Gateway URL')
    apic_service_name = utils.get_deployment_output(output, 'apicServiceName', 'API Center Name')
    acr_name = utils.get_deployment_output(output, 'acrName', 'ACR Name')
    acr_login_server = utils.get_deployment_output(output, 'acrLoginServer', 'ACR Login Server')
    aca_env_name = utils.get_deployment_output(output, 'acaEnvName', 'ACA Environment Name')
    weather_mcp_url = utils.get_deployment_output(output, 'weatherMcpUrl', 'Weather MCP URL')
    catalog_mcp_url = utils.get_deployment_output(output, 'catalogMcpUrl', 'Catalog MCP URL')
    order_mcp_url = utils.get_deployment_output(output, 'orderMcpUrl', 'Order MCP URL')
    calculator_mcp_url = utils.get_deployment_output(output, 'calculatorMcpUrl', 'Calculator MCP URL')
    app_insights_name = utils.get_deployment_output(output, 'appInsightsName', 'App Insights Name')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for sub in apim_subscriptions:
        utils.print_info(f"Subscription: {sub['name']} — Key: ****{sub['key'][-4:]}")
    api_key = apim_subscriptions[0].get("key") if apim_subscriptions else None

## Part 2 — Build & Deploy Container Images

<a id='3'></a>
### 3️⃣ Build all 4 container images with ACR Tasks

Each FastMCP server is built directly in the cloud using [ACR Tasks](https://learn.microsoft.com/azure/container-registry/container-registry-tasks-overview) — no local Docker required.

In [ ]:
# Build the first 3 MCP container images
build_configs = [
    {"name": "weather-mcp",  "context": "src/weather/container/"},
    {"name": "catalog-mcp",  "context": "src/product-catalog/container/"},
    {"name": "order-mcp",    "context": "src/place-order/container/"},
]

for config in build_configs:
    utils.run(
        f"az acr build --registry {acr_name} --resource-group {resource_group_name} --image {config['name']}:latest {config['context']}",
        f"✅ {config['name']} image built",
        f"Failed to build {config['name']} image"
    )

<a id='4'></a>
### 4️⃣ Update the first 3 Container Apps with built images

The Bicep deployment creates container apps with a placeholder image. Now we update them with the freshly built images.

In [ ]:
# Update the first 3 container apps with freshly built images
containers = [
    {"name": "weather-mcp", "image": f"{acr_login_server}/weather-mcp:latest"},
    {"name": "catalog-mcp", "image": f"{acr_login_server}/catalog-mcp:latest"},
    {"name": "order-mcp",   "image": f"{acr_login_server}/order-mcp:latest"},
]

for container in containers:
    utils.run(
        f"az containerapp update --name {container['name']} --resource-group {resource_group_name} --image {container['image']}",
        f"Updated {container['name']}",
        f"Failed to update {container['name']}"
    )

## Part 3 — Test MCP Servers

<a id='5'></a>
### 5️⃣ Test MCP servers directly (Container Apps endpoints)

Connect to each MCP server using the Python `mcp` client over Streamable HTTP to verify the tools are registered.

In [ ]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

from mcp import ClientSession
from mcp.client.streamable_http import streamable_http_client


async def list_tools(server_url: str, label: str = ""):
    """Connect to an MCP server via Streamable HTTP and list available tools."""
    display = label or server_url
    try:
        async with streamable_http_client(server_url) as (read_stream, write_stream, _):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                tools = await session.list_tools()
                tool_names = [tool.name for tool in tools.tools]
                utils.print_ok(f"{display}: {tool_names}")
                return tool_names
    except Exception as e:
        utils.print_error(f"{display}: {e}")
        return []


async def call_tool(server_url: str, tool_name: str, arguments: dict, label: str = ""):
    """Connect to an MCP server and call a specific tool."""
    display = label or server_url
    try:
        async with streamable_http_client(server_url) as (read_stream, write_stream, _):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                result = await session.call_tool(tool_name, arguments)
                utils.print_ok(f"{display} → {tool_name}({arguments})")
                for content in result.content:
                    print(f"   {content.text}")
                return result
    except Exception as e:
        utils.print_error(f"{display} → {tool_name}: {e}")
        return None

utils.print_ok("MCP test helpers ready")

In [ ]:
# Test Weather MCP directly on ACA
print("☁️ Weather MCP — Direct (ACA)")
asyncio.run(list_tools(f"{weather_mcp_url}/weather/mcp", "Weather"))
asyncio.run(call_tool(f"{weather_mcp_url}/weather/mcp", "get_cities", {"country": "uk"}, "Weather"))
asyncio.run(call_tool(f"{weather_mcp_url}/weather/mcp", "get_weather", {"city": "London"}, "Weather"))

In [ ]:
# Test Product Catalog MCP directly on ACA
print("📦 Product Catalog MCP — Direct (ACA)")
asyncio.run(list_tools(f"{catalog_mcp_url}/catalog/mcp", "Catalog"))
asyncio.run(call_tool(f"{catalog_mcp_url}/catalog/mcp", "search_products", {"query": "keyboard"}, "Catalog"))
asyncio.run(call_tool(f"{catalog_mcp_url}/catalog/mcp", "list_categories", {}, "Catalog"))

In [ ]:
# Test Order Service MCP directly on ACA
print("🛒 Order Service MCP — Direct (ACA)")
asyncio.run(list_tools(f"{order_mcp_url}/order/mcp", "Order"))
asyncio.run(call_tool(f"{order_mcp_url}/order/mcp", "place_order", {"product_id": "PROD-002", "quantity": 2}, "Order"))
asyncio.run(call_tool(f"{order_mcp_url}/order/mcp", "list_orders", {}, "Order"))

<a id='6'></a>
### 6️⃣ Test MCP servers via APIM gateway

Connect through the **API Management** gateway which proxies the MCP traffic to the container apps. The APIM URL pattern is: `{gateway}/weather-mcp/mcp`

In [ ]:
# Test Weather MCP via APIM gateway
print("☁️ Weather MCP — via APIM")
asyncio.run(list_tools(f"{apim_gateway_url}/weather-mcp/mcp", "APIM/Weather"))
asyncio.run(call_tool(f"{apim_gateway_url}/weather-mcp/mcp", "get_weather", {"city": "Manchester"}, "APIM/Weather"))

In [ ]:
# Test Product Catalog MCP via APIM gateway
print("📦 Product Catalog MCP — via APIM")
asyncio.run(list_tools(f"{apim_gateway_url}/catalog-mcp/mcp", "APIM/Catalog"))
asyncio.run(call_tool(f"{apim_gateway_url}/catalog-mcp/mcp", "get_product", {"product_id": "PROD-003"}, "APIM/Catalog"))

In [ ]:
# Test Order Service MCP via APIM gateway
print("🛒 Order Service MCP — via APIM")
asyncio.run(list_tools(f"{apim_gateway_url}/order-mcp/mcp", "APIM/Order"))
asyncio.run(call_tool(f"{apim_gateway_url}/order-mcp/mcp", "place_order", {"product_id": "PROD-001", "quantity": 3}, "APIM/Order"))

<a id='7'></a>
### 7️⃣ Test summary — first 3 MCP servers

Comprehensive test across all 3 MCP servers (direct + APIM).

In [ ]:
# Comprehensive test summary
async def run_all_tests():
    results = []
    
    test_cases = [
        # (label, url, expected_tools)
        ("☁️ Weather (Direct)",    f"{weather_mcp_url}/weather/mcp",           ["get_cities", "get_weather"]),
        ("📦 Catalog (Direct)",    f"{catalog_mcp_url}/catalog/mcp",           ["search_products", "get_product", "list_categories", "check_stock"]),
        ("🛒 Order (Direct)",      f"{order_mcp_url}/order/mcp",               ["place_order", "get_order", "list_orders"]),
        ("☁️ Weather (APIM)",      f"{apim_gateway_url}/weather-mcp/mcp",      ["get_cities", "get_weather"]),
        ("📦 Catalog (APIM)",      f"{apim_gateway_url}/catalog-mcp/mcp",      ["search_products", "get_product", "list_categories", "check_stock"]),
        ("🛒 Order (APIM)",        f"{apim_gateway_url}/order-mcp/mcp",        ["place_order", "get_order", "list_orders"]),
    ]
    
    for label, url, expected in test_cases:
        tools = await list_tools(url, label)
        passed = set(expected).issubset(set(tools))
        results.append((label, "✅ PASS" if passed else "❌ FAIL", tools))
    
    print("\n" + "=" * 60)
    print("📊 TEST SUMMARY")
    print("=" * 60)
    passed = sum(1 for _, status, _ in results if "PASS" in status)
    for label, status, tools in results:
        print(f"  {status}  {label} — {len(tools)} tools")
    print(f"\n  {passed}/{len(results)} tests passed")
    print("=" * 60)

asyncio.run(run_all_tests())

## Part 4 — Add a 4th MCP Server (Incremental Scale-Out)

<a id='8'></a>
### 8️⃣ Build & deploy the Calculator MCP container

Demonstrate how easy it is to add a new MCP server to the fleet. The **Calculator MCP** provides `calculate`, `sqrt`, and `convert_units` tools.

The container app and APIM proxy were already provisioned by the Bicep deployment — we just need to build the image and update the container.

In [ ]:
# Build the Calculator MCP container image
utils.run(
    f"az acr build --registry {acr_name} --resource-group {resource_group_name} --image calculator-mcp:latest src/calculator/container/",
    "Calculator MCP image built",
    "Failed to build Calculator MCP image"
)

# Update the calculator container app with the built image
utils.run(
    f"az containerapp update --name calculator-mcp --resource-group {resource_group_name} --image {acr_login_server}/calculator-mcp:latest",
    "Calculator MCP container updated",
    "Failed to update Calculator MCP container"
)

In [ ]:
# Test Calculator MCP — direct on ACA
print("🧮 Calculator MCP — Direct (ACA)")
asyncio.run(list_tools(f"{calculator_mcp_url}/calculator/mcp", "Calculator"))
asyncio.run(call_tool(f"{calculator_mcp_url}/calculator/mcp", "calculate", {"operation": "multiply", "a": 7, "b": 6}, "Calculator"))
asyncio.run(call_tool(f"{calculator_mcp_url}/calculator/mcp", "sqrt", {"value": 144}, "Calculator"))
asyncio.run(call_tool(f"{calculator_mcp_url}/calculator/mcp", "convert_units", {"value": 100, "from_unit": "km", "to_unit": "miles"}, "Calculator"))

# Test Calculator MCP — via APIM gateway
print("\n🧮 Calculator MCP — via APIM")
asyncio.run(list_tools(f"{apim_gateway_url}/calculator-mcp/mcp", "APIM/Calculator"))
asyncio.run(call_tool(f"{apim_gateway_url}/calculator-mcp/mcp", "calculate", {"operation": "add", "a": 100, "b": 200}, "APIM/Calculator"))

## Part 5 — Discover MCP Servers via API Center

<a id='9'></a>
### 9️⃣ Browse the API Center catalog

All 4 MCP servers are registered in **Azure API Center** as discoverable APIs with `kind: mcp`. Each registration includes:
- **API entry** with title, description, and lifecycle state
- **Version** (1.0.0) and **definition**
- **Deployment** with the APIM gateway `runtimeUri`

This enables developers and AI agents to discover available MCP servers programmatically.

In [ ]:
# List all MCP APIs registered in API Center
output = utils.run(
    f"az apic api list --resource-group {resource_group_name} --service-name {apic_service_name} --output table",
    "Listed APIs in API Center",
    "Failed to list APIs"
)

In [ ]:
# Search for MCP servers by kind and show deployments with runtime URIs
mcp_servers_found = []

output = utils.run(
    f'az apic api list --resource-group {resource_group_name} --service-name {apic_service_name} --query "[?kind==\'mcp\']" --output json',
    "Filtered MCP-kind APIs",
    "Failed to filter APIs"
)

if output.success and output.json_data:
    print(f"\n🔍 Found {len(output.json_data)} MCP servers in API Center:\n")
    for api in output.json_data:
        api_name = api.get('name', 'unknown')
        title = api.get('title', api_name)
        description = api.get('summary', api.get('description', ''))
        lifecycle = api.get('lifecycleState', 'unknown')
        
        # Get the deployment for this API to find the runtime URI
        deploy_output = utils.run(
            f"az apic api deployment list --resource-group {resource_group_name} --service-name {apic_service_name} --api-id {api_name} --output json",
            "", ""
        )
        
        runtime_uri = ""
        if deploy_output.success and deploy_output.json_data and len(deploy_output.json_data) > 0:
            server = deploy_output.json_data[0].get('server', {})
            uris = server.get('runtimeUri', [])
            runtime_uri = uris[0] if uris else ""
        
        mcp_servers_found.append({"name": api_name, "title": title, "uri": runtime_uri})
        
        print(f"  📡 {title}")
        print(f"     Kind: mcp | Lifecycle: {lifecycle}")
        print(f"     Description: {description}")
        print(f"     Gateway URI: {runtime_uri}")
        print()

In [ ]:
# Dynamically connect to all discovered MCP servers and list their tools
print("🔗 Connecting to discovered MCP servers via API Center...\n")

for server in mcp_servers_found:
    if server['uri']:
        mcp_url = f"{server['uri']}/mcp"
        tools = asyncio.run(list_tools(mcp_url, f"  {server['title']}"))

print(f"\n✅ All {len(mcp_servers_found)} MCP servers are discoverable and reachable via API Center")

## Part 6 — Monitoring, Logging & Security

<a id='10'></a>
### 🔟 Observe MCP traffic in Application Insights

Every MCP API in APIM has **Application Insights diagnostics** configured with:
- **Verbose** logging — captures all requests and errors
- **W3C** correlation — distributed tracing across services
- **100% sampling** — every MCP call is recorded
- **Metrics** — request counts, latency, error rates

Use the [MCP Insights Dashboard](https://portal.azure.com) in the Azure Portal or query logs directly.

In [ ]:
# Generate some MCP traffic for monitoring (call each server through APIM)
print("📊 Generating MCP traffic for Application Insights...\n")

traffic_calls = [
    (f"{apim_gateway_url}/weather-mcp/mcp",     "get_weather",      {"city": "London"}),
    (f"{apim_gateway_url}/weather-mcp/mcp",     "get_cities",       {"country": "portugal"}),
    (f"{apim_gateway_url}/catalog-mcp/mcp",     "search_products",  {"query": "mouse"}),
    (f"{apim_gateway_url}/catalog-mcp/mcp",     "check_stock",      {"product_id": "PROD-001"}),
    (f"{apim_gateway_url}/order-mcp/mcp",       "place_order",      {"product_id": "PROD-006", "quantity": 1}),
    (f"{apim_gateway_url}/calculator-mcp/mcp",  "calculate",        {"operation": "divide", "a": 100, "b": 7}),
    (f"{apim_gateway_url}/calculator-mcp/mcp",  "convert_units",    {"value": 72, "from_unit": "fahrenheit", "to_unit": "celsius"}),
]

for url, tool, args in traffic_calls:
    asyncio.run(call_tool(url, tool, args, f"APIM"))

utils.print_ok(f"Generated {len(traffic_calls)} MCP calls through APIM — check Application Insights for traces")

In [ ]:
# Query Application Insights for recent MCP requests
import time
print("⏳ Waiting 30 seconds for telemetry to propagate...\n")
time.sleep(30)

# Query App Insights for MCP API request logs
query = "requests | where timestamp > ago(10m) | where url contains 'mcp' | summarize count() by name, resultCode | order by count_ desc"
output = utils.run(
    f'az monitor app-insights query --app {app_insights_name} --resource-group {resource_group_name} --analytics-query "{query}" --output json',
    "Retrieved Application Insights telemetry",
    "Failed to query Application Insights (telemetry may still be ingesting)"
)

if output.success and output.json_data:
    tables = output.json_data.get('tables', [])
    if tables and tables[0].get('rows'):
        print("\n📈 MCP API Request Summary (last 10 minutes):\n")
        print(f"  {'API Name':<40} {'Status':<10} {'Count':<10}")
        print(f"  {'-'*40} {'-'*10} {'-'*10}")
        for row in tables[0]['rows']:
            print(f"  {row[0]:<40} {row[1]:<10} {row[2]:<10}")
    else:
        utils.print_info("No telemetry data yet — it may take a few minutes to appear")

## Part 7 — End-to-End Agent Workflow

<a id='11'></a>
### 1️⃣1️⃣ Multi-tool agent using discovered MCP servers

Demonstrate an end-to-end workflow where an agent:
1. **Discovers** MCP servers from API Center
2. **Connects** to multiple MCP servers simultaneously
3. **Chains** tool calls across servers — search catalog → check stock → place order → calculate total

In [ ]:
# End-to-end agent workflow: Discover → Connect → Chain tools
async def agent_workflow():
    """Simulate an AI agent that discovers and chains MCP tools."""
    
    print("🤖 Agent Workflow: 'Find a keyboard, check if it's in stock, order 2, and calculate the total in GBP'\n")
    print("=" * 70)
    
    # Step 1: Discover MCP servers from API Center
    print("\n📡 Step 1 — Discover MCP servers from API Center")
    for server in mcp_servers_found:
        print(f"   Found: {server['title']} → {server['uri']}")
    
    # Step 2: Search the catalog for keyboards
    print("\n🔍 Step 2 — Search product catalog for 'keyboard'")
    result = await call_tool(f"{apim_gateway_url}/catalog-mcp/mcp", "search_products", {"query": "keyboard"}, "Agent→Catalog")
    
    # Step 3: Check stock
    print("\n📦 Step 3 — Check stock for PROD-002 (Mechanical Keyboard)")
    result = await call_tool(f"{apim_gateway_url}/catalog-mcp/mcp", "check_stock", {"product_id": "PROD-002"}, "Agent→Catalog")
    
    # Step 4: Place order for 2 keyboards
    print("\n🛒 Step 4 — Place order: 2× Mechanical Keyboard")
    result = await call_tool(f"{apim_gateway_url}/order-mcp/mcp", "place_order", {"product_id": "PROD-002", "quantity": 2}, "Agent→Order")
    
    # Step 5: Calculate total and convert to GBP
    print("\n🧮 Step 5 — Calculate total: 89.99 × 2")
    result = await call_tool(f"{apim_gateway_url}/calculator-mcp/mcp", "calculate", {"operation": "multiply", "a": 89.99, "b": 2}, "Agent→Calculator")
    
    # Step 6: Check today's weather for delivery
    print("\n☁️ Step 6 — Check weather at delivery location")
    result = await call_tool(f"{apim_gateway_url}/weather-mcp/mcp", "get_weather", {"city": "London"}, "Agent→Weather")
    
    print("\n" + "=" * 70)
    print("✅ Agent workflow complete — chained 5 tools across 4 MCP servers!")
    print("   All traffic routed through APIM gateway, logged in Application Insights")

asyncio.run(agent_workflow())

<a id='12'></a>
### 1️⃣2️⃣ Final comprehensive test — all 4 MCP servers (8 tests)

In [ ]:
# Final comprehensive test — all 4 MCP servers, direct + APIM
async def run_final_tests():
    results = []
    
    test_cases = [
        ("☁️ Weather (Direct)",       f"{weather_mcp_url}/weather/mcp",             ["get_cities", "get_weather"]),
        ("📦 Catalog (Direct)",       f"{catalog_mcp_url}/catalog/mcp",             ["search_products", "get_product", "list_categories", "check_stock"]),
        ("🛒 Order (Direct)",         f"{order_mcp_url}/order/mcp",                 ["place_order", "get_order", "list_orders"]),
        ("🧮 Calculator (Direct)",    f"{calculator_mcp_url}/calculator/mcp",       ["calculate", "sqrt", "convert_units"]),
        ("☁️ Weather (APIM)",         f"{apim_gateway_url}/weather-mcp/mcp",        ["get_cities", "get_weather"]),
        ("📦 Catalog (APIM)",         f"{apim_gateway_url}/catalog-mcp/mcp",        ["search_products", "get_product", "list_categories", "check_stock"]),
        ("🛒 Order (APIM)",           f"{apim_gateway_url}/order-mcp/mcp",          ["place_order", "get_order", "list_orders"]),
        ("🧮 Calculator (APIM)",      f"{apim_gateway_url}/calculator-mcp/mcp",     ["calculate", "sqrt", "convert_units"]),
    ]
    
    for label, url, expected in test_cases:
        tools = await list_tools(url, label)
        passed = set(expected).issubset(set(tools))
        results.append((label, "✅ PASS" if passed else "❌ FAIL", tools))
    
    print("\n" + "=" * 70)
    print("📊 FINAL TEST SUMMARY — ALL 4 MCP SERVERS")
    print("=" * 70)
    passed = sum(1 for _, status, _ in results if "PASS" in status)
    for label, status, tools in results:
        print(f"  {status}  {label:<30} — {len(tools)} tools: {tools}")
    print(f"\n  {passed}/{len(results)} tests passed")
    
    # Summary of what was demonstrated
    print("\n" + "=" * 70)
    print("📋 DEMO SUMMARY")
    print("=" * 70)
    print("  ✅ 4 FastMCP servers deployed as Azure Container Apps")
    print("  ✅ APIM gateway proxying all MCP servers (Streamable HTTP)")
    print("  ✅ API Center registration for MCP discoverability")
    print("  ✅ Application Insights diagnostics (verbose, W3C, 100% sampling)")
    print("  ✅ MCP Insights Dashboard deployed to Azure Portal")
    print("  ✅ End-to-end agent workflow chaining tools across 4 servers")
    print("=" * 70)

asyncio.run(run_final_tests())

## Part 8 — Clean up

<a id='cleanup'></a>
### 🗑️ Delete all resources

⚠️ Running this cell will permanently delete the resource group and all deployed resources (APIM, ACA, ACR, API Center, etc.).

In [ ]:
# Delete the resource group and all resources within it
utils.run(
    f"az group delete --name {resource_group_name} --yes --no-wait",
    f"Resource group '{resource_group_name}' deletion initiated",
    f"Failed to delete resource group '{resource_group_name}'"
)